In [6]:
#%pylab inline
#get epochs using your mne-python pipeline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import sys
import os

import numpy as np
from sklearn.preprocessing import MinMaxScaler
import mne

#os.chdir('/m/nbe/project/rtmeg/problearn/mneflow/')
os.chdir('/u/20/vranoug1/unix/OPM-BCI/mneflow-dev/mneflow/')
import mneflow
from mneflow import keras_models

from mneflow.keras_utils import plot_metrics, plot_output, plot_history_v2
from mneflow.keras_utils import r_square, soft_acc, rmse

# Force enable eager execution after importing mneflow
tf.compat.v1.enable_eager_execution()
mne.set_log_level(verbose='CRITICAL')


path = '/m/nbe/work/vranoug1/OPM-BCI/datasets/'
fname = 'sim_2_erps_50nA.npz'


import_opt = dict(fs=500,
                  savepath='../tfr/',
                  out_name=fname+'0',
                  input_type='trials',
                  target_type='float',
                  val_size=0.2,
                  array_keys={'X': 'X', 'y': 'y'},
                  # picks=None, target_picks=None,
                  scale=True,
                  scale_interval=None,
                  # crop_baseline=True,
                  decimate=None,
                  bp_filter=False,
                  overwrite=False,
                  test_set='holdout')

meta = mneflow.produce_tfrecords([path+fname], **import_opt)
dataset = mneflow.Dataset(meta, train_batch=100, class_subset=None, pick_channels=None, decim=None)

Metadata file found, restoring


In [7]:
sys.path
print('---------\n'+mneflow.__file__)
print('tf version: '+tf.__version__)
print('executing eagerly: '+str(tf.executing_eagerly())+'\n---------')

---------
/u/20/vranoug1/unix/.conda/envs/py3ml2/lib/python3.7/site-packages/mneflow/__init__.py
tf version: 2.0.0
executing eagerly: True
---------


In [8]:
#specify optimizer parmeters
optim = tf.keras.optimizers.Adam(learning_rate=3e-4)

# specify model parameters
graph_specs = dict(n_ls=64,  # number of latent factors
                   filter_length=17,  # convolutional filter length
                   pooling=5,  # convlayer pooling factor
                   stride=5,  # stride parameter for pooling layer
                   padding='SAME',
                   dropout=.5,
                   nonlin=tf.nn.relu,
                   pool_type='max',
                   out_dim=np.prod(meta['y_shape']),
                   axis=1,
                   y_shape=meta['y_shape'],
                   model_path = import_opt['savepath'],  # not used at the moment
                   # regularization parameters
                   l1=3e-4,
                   l2=0,
                   # LSTM parameters
                   rnn_units=np.prod(meta['y_shape']),
                   rnn_dropout=0.0,
                   rnn_nonlin='tanh',
                   rnn_rec_nonlin='tanh',
                   rnn_forget_bias=True,
                   rnn_seq=True,
                   unroll=False)

# 1. LFLSTM

In [9]:
model = keras_models.LFLSTM(graph_specs, dataset)
loss_f = tf.compat.v1.losses.mean_squared_error

# % builtin
model.compile(loss=loss_f, optimizer=optim, metrics=[r_square, soft_acc, rmse])

de-mix init : OK
lstm init : OK
fc init : OK


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=6)

#train the model
history = model.fit(dataset.train, validation_data=dataset.val, epochs=30000, steps_per_epoch=100,
                    validation_steps=1, callbacks=[stop_early], verbose=1)

[None, 102, 300]
input x0 (?, 102, 300, 1)
input x0 (?, 102, 300, 1)
de-mix built : OK
dmx (?, 300, 1, 64)
demix dmx (?, 300, 1, 64)
reshaped dmx (?, 300, 64, 1)
conv build : OK
varconv (?, 60, 64, 1)
reshaped varconv (?, 1, 3840)
lstm (?, 1, 64)
fc ::: 64 1
fc build : OK
fc y_ (?, 1)
Train for 100 steps, validate for 1 steps
Epoch 1/30000
[None, 102, 300]
input x0 (?, 102, 300, 1)
input x0 (?, 102, 300, 1)
dmx (?, 300, 1, 64)
demix dmx (?, 300, 1, 64)
reshaped dmx (?, 300, 64, 1)
varconv (?, 60, 64, 1)
reshaped varconv (?, 1, 3840)
lstm (?, 1, 64)
fc y_ (?, 1)
[None, 102, 300]
input x0 (?, 102, 300, 1)
input x0 (?, 102, 300, 1)
demix dmx (?, 300, 1, 64)
reshaped dmx (?, 300, 64, 1)
varconv (?, 60, 64, 1)
reshaped varconv (?, 1, 3840)
lstm (?, 1, 64)
fc y_ (?, 1)
 99/100 [============================>.] - ETA: 0s - loss: 6.2647 - r_square: -22.6590 - soft_acc: 0.0023 - rmse: 1.0741[None, 102, 300]
input x0 (?, 102, 300, 1)
input x0 (?, 102, 300, 1)
dmx (?, 300, 1, 64)
demix dmx (?, 300

100/100 [==============================] - 65s 648ms/step - loss: 0.3781 - r_square: -0.3673 - soft_acc: 0.0135 - rmse: 0.2796 - val_loss: 0.3500 - val_r_square: -0.0658 - val_soft_acc: 0.0000e+00 - val_rmse: 0.2637
Epoch 33/30000
 72/100 [====================>.........] - ETA: 18s - loss: 0.3670 - r_square: -0.5183 - soft_acc: 0.0142 - rmse: 0.2927

In [ ]:
model.summary()
results = model.evaluate(dataset.test, steps=1, verbose=1)
plot_history_v2(history, title=model.name.upper(), nepochs=0, nend=None)

# 2. LFCNN

In [ ]:
model = keras_models.LFCNN(graph_specs, dataset)
loss_f = tf.compat.v1.losses.mean_squared_error

# % builtin
model.compile(loss=loss_f, optimizer=optim, metrics=[r_square, soft_acc, rmse])


stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=6)

#train the model
history = model.fit(dataset.train, validation_data=dataset.val, epochs=30000, steps_per_epoch=100,
                    validation_steps=1, callbacks=[stop_early], verbose=1)

In [ ]:
model.summary()
results = model.evaluate(dataset.test, steps=1, verbose=1)
plot_history_v2(history, title=model.name.upper(), nepochs=0, nend=None)

# 3. VARCNN

In [ ]:
model = keras_models.VARCNN(graph_specs, dataset)
loss_f = tf.compat.v1.losses.mean_squared_error

# % builtin
model.compile(loss=loss_f, optimizer=optim, metrics=[r_square, soft_acc, rmse])


stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=6)

#train the model
history = model.fit(dataset.train, validation_data=dataset.val, epochs=30000, steps_per_epoch=100,
                    validation_steps=1, callbacks=[stop_early], verbose=1)

In [ ]:
model.summary()
results = model.evaluate(dataset.test, steps=1, verbose=1)
plot_history_v2(history, title=model.name.upper(), nepochs=0, nend=None)

# 4. VARLSTM

In [ ]:
model = keras_models.VARLSTM(graph_specs, dataset)
loss_f = tf.compat.v1.losses.mean_squared_error

# % builtin
model.compile(loss=loss_f, optimizer=optim, metrics=[r_square, soft_acc, rmse])


stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=6)

#train the model
history = model.fit(dataset.train, validation_data=dataset.val, epochs=30000, steps_per_epoch=100,
                    validation_steps=1, callbacks=[stop_early], verbose=1)

In [ ]:
model.summary()
results = model.evaluate(dataset.test, steps=1, verbose=1)
plot_history_v2(history, title=model.name.upper(), nepochs=0, nend=None)